In [1]:
import Auto_paint_self

In [2]:
CN05_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\CN05.1_Pre_1961_2021_daily_025x025.nc','pre','yes','time','2001-01-01','2017-10-19','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
predicty,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one','E:/xuanxuan/unet_pre_predicty_CN05_morev.nc','predicty','yes','time','2017-10-20','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
testy,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one','E:/xuanxuan/unet_pre_testy_CN05_morev.nc','testy','yes','time','2017-10-20','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')

F:\anaconda\envs\tensorflow\lib\site-packages\xarray\coding\times.py:170: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [4]:
import numpy as np
CN05_PRE=np.nan_to_num(CN05_PRE,nan=0)

In [5]:
#CDF匹配
def Auto_cdf_matching(vx,vy):
    import numpy as np
    from scipy.interpolate import InterpolatedUnivariateSpline
    from scipy.optimize import curve_fit

    if np.array(vx).ndim==1:
        vx_cdf = (np.arange(len(vx)) +  1) / (len(vx))
        vy_cdf = (np.arange(len(vy)) +  1) / (len(vy))
        
        spl = InterpolatedUnivariateSpline(vx_cdf, np.sort(vx))
        vx_interp = spl(vy_cdf)
        
        def func(x, a, b, c, d):
            return a*x + b*x**2 + c*x**3 + d
        
        popt = curve_fit(func, vx_interp, np.sort(vy))[0]
        
        matched_vx = func(vx, *popt)
    elif np.array(vx).ndim==2:
        matched_vx=np.zeros((vx.shape[0],vx.shape[1]))
        for i in range(vx.shape[1]):
            vx_cdf = (np.arange(len(vx[:,i])) +  1) / (len(vx[:,i]))
            vy_cdf = (np.arange(len(vy[:,i])) +  1) / (len(vy[:,i]))
            
            spl = InterpolatedUnivariateSpline(vx_cdf, np.sort(vx[:,i]))
            vx_interp = spl(vy_cdf)
            
            def func(x, a, b, c, d):
                return a*x + b*x**2 + c*x**3 + d
            
            popt = curve_fit(func, vx_interp, np.sort(vy[:,i]))[0]
            
            matched_vx[:,i] = func(vx[:,i], *popt)
    elif np.array(vx).ndim==3:
        matched_vx=np.zeros((vx.shape[0],vx.shape[1],vx.shape[2]))
        for i in range(vx.shape[1]):
            for j in range(vx.shape[2]):
                vx_cdf = (np.arange(len(vx[:,i,j])) +  1) / (len(vx[:,i,j]))
                vy_cdf = (np.arange(len(vy[:,i,j])) +  1) / (len(vy[:,i,j]))
                
                spl = InterpolatedUnivariateSpline(vx_cdf, np.sort(vx[:,i,j]))
                vx_interp = spl(vy_cdf)
                
                def func(x, a, b, c, d):
                    return a*x + b*x**2 + c*x**3 + d
                
                popt = curve_fit(func, vx_interp, np.sort(vy[:,i,j]))[0]
                
                matched_vx[:,i,j] = func(vx[:,i,j], *popt)
    elif np.array(vx).ndim==4:
        matched_vx=np.zeros((vx.shape[0],vx.shape[1],vx.shape[2],vx.shape[3]))
        for i in range(vx.shape[1]):
            for j in range(vx.shape[2]):
                for k in range(vx.shape[3]):
                    vx_cdf = (np.arange(len(vx[:,i,j,k])) +  1) / (len(vx[:,i,j,k]))
                    vy_cdf = (np.arange(len(vy[:,i,j,k])) +  1) / (len(vy[:,i,j,k]))
                    
                    spl = InterpolatedUnivariateSpline(vx_cdf, np.sort(vx[:,i,j,k]))
                    vx_interp = spl(vy_cdf)
                    
                    def func(x, a, b, c, d):
                        return a*x + b*x**2 + c*x**3 + d
                    
                    popt = curve_fit(func, vx_interp, np.sort(vy[:,i,j,k]))[0]
                    
                    matched_vx[:,i,j,k] = func(vx[:,i,j,k], *popt)

    return matched_vx

In [6]:
from sklearn.model_selection import train_test_split
import Auto_paint_self
import numpy as np
predicty=Auto_cdf_matching(np.array(predicty),CN05_PRE[np.random.randint(0,predicty.shape[0], predicty.shape[0]),:,:])

C:\Users\TBYC\AppData\Local\Temp\ipykernel_181744\2651361597.py:48: OptimizeWarning: Covariance of the parameters could not be estimated
  popt = curve_fit(func, vx_interp, np.sort(vy[:,i,j]))[0]


In [7]:
Auto_paint_self.create_nc(times,lat,lon,None,predicty,'time','lat','lon','predicty','no',None,'yes','yes','yes','E:/xuanxuan/unet_pre_predicty_CN05_morev_cdf.nc')

In [8]:
CN05_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\CN05.1_Pre_1961_2021_daily_025x025.nc','pre','yes','time','2017-10-20','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')

F:\anaconda\envs\tensorflow\lib\site-packages\xarray\coding\times.py:170: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [9]:
import warnings

warnings.filterwarnings("ignore")
r=np.zeros((testy.shape[1],testy.shape[2]))
p=np.zeros((testy.shape[1],testy.shape[2]))
from scipy.stats import pearsonr
for i in range(testy.shape[1]):
    for j in range(testy.shape[2]):
        if np.isnan(CN05_PRE[:,i,j]).any():
            r[i,j],p[i,j]=np.nan,np.nan
        else:
            r[i,j],p[i,j] = pearsonr(predicty[:,i,j],testy[:,i,j])
print(np.nanmean(r))

0.6800661733267042
